<a href="https://colab.research.google.com/github/caumannerman/cg_project/blob/main/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# 라이브러리 설치
!pip install -U finance-datareader
!pip install beautifulsoup4
!pip install pykrx
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
from pykrx import stock
import requests
from io import BytesIO
from google.colab import data_table



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip

In [28]:
# 헤더 부분에 리퍼러(Referer)를 추가
# 리퍼러란 링크를 통해서 각각의 웹사이트로 방문할 때 남은 흔적(로봇으로 인식하지 않기 위해)
# generate.cmd에서 Request URL 가져오기
gen_otp_url = 'https://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# download.cmd에서 General의 Request URL 부분
down_url = 'https://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
#########   여기까지가 변하지 않는 것들     #########

otp = requests.post(gen_otp_url, gen_otp_data, headers=headers).text



# # requests의 post 함수를 사용 해당 url에 접속하여 otp코드 제출
# down_sector_KS = requests.post(down_url, {'code': otp}, headers=headers)

# # 다운 받은 csv파일을 pandas의 read_csv 함수로 불러오기
# # read_csv 함수의 argment에 적합할 수 있도록 ByteIO 함수를 이용하여 바이너리 스트림 형태로 만든다
# sector_KS = pd.read_csv(BytesIO(down_sector_KS.content), encoding='EUC-KR')

# generate.cmd에서 Form Data와 동일
# Total_data=pd.DataFrame()

# headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'} 

In [29]:
# 이곳에 업종이름을 Key로 하여 빈 배열을 value로 갖는 딕셔너리를 만들어야함.
sector_list = set()
# 2022년 1월 1일 기준으로 존재하는 모든 업종을 sector_list에 담을 것이다. 
# 아래의 기간만 늘리면, 더 넓은 기간의 업종명을 담을 수 있다.
for i in range(20220101,20220131):
    gen_otp_data={
        'locale': 'ko_KR',
        'mktId': 'STK',
        'trdDd': str(i),
        'money': '1',
        'csvxls_isNo': 'false',
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
    }
    otp = requests.post(gen_otp_url, gen_otp_data, headers=headers).text
    down_sector_KS = requests.post(down_url, {'code': otp}, headers=headers)
    # 이것이 해당 일자의 DataFrame
    sector_KS = pd.read_csv(BytesIO(down_sector_KS.content), encoding='EUC-KR')
    # NaN을 제거
    sector_KS = sector_KS.dropna()
    # sector_data=sector_KS[["종목코드", "종목명","업종명","등락률"]]
    sector_data=sector_KS[["업종명"]]
    # 하루 데이터를 갖고올 때마다, data 길이만큼 다 돌며 업종명을 다 set에 추가한다.
    for i in range(len(sector_data)):
      sector_list.add(sector_data.iloc[i]['업종명'])
sector_list = list(sector_list)
print(sector_list)

['의약품', '기타제조업', '유통업', '종이목재', '농업, 임업 및 어업', '전기전자', '증권', '기계', '음식료품', '운수장비', '화학', '기타금융', '섬유의복', '은행', '광업', '운수창고업', '보험', '전기가스업', '서비스업', '철강금속', '통신업', '비금속광물', '의료정밀', '건설업']


In [30]:
print(len(sector_list))

24


In [34]:
# sector_list를 사용하여 하루마다의 업종별 등락률 평균을 담을 dictionary를 만든다
total_fluc_rate_dict = dict()

In [35]:
# total_fluc_rate_dict에 각 업종 이름을 key로 하여 빈 배열을 value로 초기화한다
for i in sector_list:
  total_fluc_rate_dict[i] = []
print(total_fluc_rate_dict)

{'의약품': [], '기타제조업': [], '유통업': [], '종이목재': [], '농업, 임업 및 어업': [], '전기전자': [], '증권': [], '기계': [], '음식료품': [], '운수장비': [], '화학': [], '기타금융': [], '섬유의복': [], '은행': [], '광업': [], '운수창고업': [], '보험': [], '전기가스업': [], '서비스업': [], '철강금속': [], '통신업': [], '비금속광물': [], '의료정밀': [], '건설업': []}


In [36]:
# 20221201~20221231까지의 매일 각 기업의 '종목코드','종목명','업종명','등락률'을 표시하여 sector_data로 print
# NaN 데이터가 있는 열을 dropna.()로 지웠음
for i in range(20221201,20221231):
    gen_otp_data={
        'locale': 'ko_KR',
        'mktId': 'STK',
        'trdDd': str(i),
        'money': '1',
        'csvxls_isNo': 'false',
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
    }
    otp = requests.post(gen_otp_url, gen_otp_data, headers=headers).text
    down_sector_KS = requests.post(down_url, {'code': otp}, headers=headers)
    # 이것이 해당 일자의 DataFrame
    sector_KS = pd.read_csv(BytesIO(down_sector_KS.content), encoding='EUC-KR')
    
    # NaN을 제거
    sector_KS = sector_KS.dropna(axis=0)
    # 주말같은 경우 빈 DataFrame이 된다. 이럴 때는 continue해서 넘어가야함  
    if len(sector_KS) == 0:
      continue
    # sector_data=sector_KS[["종목코드", "종목명","업종명","등락률"]]
    sector_data = sector_KS[["업종명","등락률"]]
    # print(sector_data)
    
    # print(sector_data)
    # for 문을 하나 더 써서,여기서 이제 업종명 별 등락률 다 더해서 평균내고, total_fluc_rate_dict에 하나씩 담는다.
    for j in range(len(sector_list)):
      now_sector = sector_data.loc[sector_data["업종명"] == sector_list[j] , :]
      now_avg = now_sector[["등락률"]].mean()
      total_fluc_rate_dict[sector_list[j]].append(now_avg[0])
    print(total_fluc_rate_dict)
    print()
    print()


    # 


    
    #sector_data
    #print(sector_data)

    # 윗줄까지 해서, sector_KS는 하루치/ 당일 상장되어있는 모든 종목의 정보 다 담겨있음 - 완료
    # 각 업종 별 평균 구해서 / 각 업종 별 일별 등락율 평균 담을 배열에 추가
    #a b c d
    #for j in (len(업종 다 들어있는 리스트)): 
    #one_month_data_dict["서비스업"].append(방금 구한 평균)
    #Total_data = pd.concat([Total_data, sector_KS], ignore_index=False)

{'의약품': [1.82], '기타제조업': [1.2013333333333334], '유통업': [1.166716417910448], '종이목재': [0.43549999999999994], '농업, 임업 및 어업': [0.53], '전기전자': [0.5065789473684209], '증권': [1.696774193548387], '기계': [0.48738095238095236], '음식료품': [1.0230434782608697], '운수장비': [0.5933846153846154], '화학': [0.9806611570247935], '기타금융': [1.028241758241758], '섬유의복': [0.7831999999999999], '은행': [2.6833333333333336], '광업': [3.32], '운수창고업': [1.1771428571428573], '보험': [1.5306666666666666], '전기가스업': [0.21400000000000005], '서비스업': [0.9767999999999999], '철강금속': [1.0934615384615385], '통신업': [0.07400000000000002], '비금속광물': [1.1092307692307692], '의료정밀': [2.39], '건설업': [6.967631578947368]}


{'의약품': [1.82, -0.46924528301886786], '기타제조업': [1.2013333333333334, -0.05266666666666673], '유통업': [1.166716417910448, -0.6243283582089553], '종이목재': [0.43549999999999994, 0.4745000000000001], '농업, 임업 및 어업': [0.53, 0.48750000000000004], '전기전자': [0.5065789473684209, -0.6028947368421054], '증권': [1.696774193548387, -0.8796774193548387], '기계'

KeyboardInterrupt: ignored

In [ ]:
#1 sector_list라는 집합을 형성
#2 sector_data의 길이만큼 반복해서 sector_data의 '업종명'만 추출
#3 sector_list는 현재 집합이라 '업종명'이 겹치는 것은 1개의 원소로 추출
#4 sector_list를 list화 시킴. sector_list의 길이는 '업종명' 개수
#5 sector_list print
sector_list = set()
for i in range(len(sector_data)):
  sector_list.add(sector_data.iloc[i]['업종명'])
sector_list = list(sector_list)
print(sector_list)

In [ ]:
#1 aaa라는 딕셔너리를 형성
#2 sector_list 길이만큼 반복하여 aaa 딕셔너리에 '업종명': value[]를 넣음
aaa = {}
print(type(aaa))
for i in range(len(sector_list)):
  aaa[sector_list[i]] = []
print(aaa)

In [ ]:
#1 sector_avg라는 딕셔너리를 형성
#2 sector_avg라는 딕셔너리에 sector_list에 있는 업종명을 넣어서 데이터 프레임을 형성. sector_list의 업종명 개수만큼 반복
#3 sector_data의 행 중 "업종명"을 sector_list에 대응시킨 후 now_sector로 선언
#4 now_sector의 '등락률'의 평균을 낸 것을 avg_result로 선언
#5 sector_avg의 업종별 key 값에 avg_result 값을 대응시켜서 딕셔너리 형성
sector_avg = dict()
for i in range(len(sector_list)):
  sector_avg[sector_list[i]] = []

for i in range(len(sector_list)):
  now_sector = sector_data.loc[sector_data["업종명"] == sector_list[i] , :]
  avg_result = now_sector[["등락률"]].mean()
  sector_avg[sector_list[i]].append(avg_result[0])
  print(sector_avg)

In [ ]:
tp = sector_data.loc[sector_data['업종명'] == "운수창고업" , :]
print(type(tp))
# print(tp)
print(len(tp))
print(tp[["등락률"]].mean())
# tp[["등락률"]].iloc[0]

In [ ]:
elec=sector_data.loc[sector_data['업종명']=="전기전자", :]
elec[["등락률"]].iloc[0]

In [ ]:
sum = 0
for i in range(len(tp[["등락률"]])):
  sum += tp[["등락률"]].iloc[i]
avg_tp=sum / len(tp[["등락률"]])
avg_tp

In [ ]:
sum = 0
for i in range(len(tp[["등락률"]])):
  sum += tp[["등락률"]].iloc[i]
avg_tp=sum / len(tp[["등락률"]])
avg_tp

In [ ]:
tp = sector_data.loc[sector_data['업종명'] == "" , :]
tp[["등락률"]].iloc[0]

In [ ]:
def calc_average(dataFrame):
  return 평균값